In [68]:
import pandas as pd
import tensorflow
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import datetime

# for jupyter
%matplotlib inline

In [69]:
dt = pd.read_csv('train.csv')
print(783/len(dt))
dt.head()

0.01804147465437788


,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,30669,Male,3.0,0,0,No,children,Rural,95.12,18.0,NaN,0
1,30468,Male,58.0,1,0,Yes,Private,Urban,87.96,39.2,never smoked,0
2,16523,Female,8.0,0,0,No,Private,Urban,110.89,17.6,NaN,0
3,56543,Female,70.0,0,0,Yes,Private,Rural,69.04,35.9,formerly smoked,0
4,46136,Male,14.0,0,0,No,Never_worked,Rural,161.28,19.1,NaN,0


In [98]:
def Normaliza_dados(dados):
    d = dados.copy()
    
    # d = dados[dados['gender'] != 'Other'].copy()
    # d = d[d['age'] >= 18]
    
    
    #d = d[d['work_type'] != "Never_worked" ]
    d.drop('smoking_status',axis=1,inplace=True)


    
    sim_nao = ['yes','no']
    residencia = ['Urban', 'Rural']
    smokes = ['formerly smoked', 'never smoked', 'smokes', 'Unknown']



    def set_gender(x):
        sexo = ['Male', 'Female','Other']
        r = sexo.index(x)
        if r == 2:
            r = 0.5
        return r
    
    d['gender'] = d['gender'].apply(set_gender)

    d['ever_married'] = d['ever_married'].apply( lambda x: sim_nao.index(x.lower()))
    d['Residence_type'] = d['Residence_type'].apply( lambda x: residencia.index(x))

    lista = ['Private', 'Self-employed', 'Govt_job', 'Never_worked']
    #lista = ['Private', 'Self-employed', 'Govt_job']
    for i in lista:
        d['work_type_'+i] = dt['work_type'].apply(lambda x: 1 if x == i else 0)
    #d['work'] = d['work_type'].apply(lambda x: lista.index(x))
    d = d.drop('work_type',axis=1)

    d['age'] = d['age'].apply(lambda x: x/100)
    d['avg_glucose_level'] = d['avg_glucose_level'].apply(lambda x: (x-55)/(300 - 55) )


    media_imc = d['bmi'].mean()
    d['bmi'] = d['bmi'].apply(lambda x: media_imc if (str(x) == 'nan') else x)

    d['bmi'] = d['bmi']/100

    return d
d = Normaliza_dados(dt)
d

,id,gender,age,hypertension,heart_disease,ever_married,Residence_type,avg_glucose_level,bmi,stroke,work_type_Private,work_type_Self-employed,work_type_Govt_job,work_type_Never_worked
0,30669,0.0,0.03,0,0,1,1,0.163755,0.180,0,0,0,0,0
1,30468,0.0,0.58,1,0,0,0,0.134531,0.392,0,1,0,0,0
2,16523,1.0,0.08,0,0,1,0,0.228122,0.176,0,1,0,0,0
3,56543,1.0,0.70,0,0,0,1,0.057306,0.359,0,1,0,0,0
4,46136,0.0,0.14,0,0,1,1,0.433796,0.191,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43395,56196,1.0,0.10,0,0,1,0,0.014857,0.204,0,0,0,0,0
43396,5450,1.0,0.56,0,0,0,0,0.647388,0.554,0,0,0,1,0
43397,28375,1.0,0.82,1,0,0,0,0.150776,0.289,0,1,0,0,0
43398,27973,0.0,0.40,0,0,0,0,0.180245,0.332,0,1,0,0,0


In [99]:
def strokes(x,v = 1):
    r = x[x['stroke'] == v ]
   # print('strokes positivados:',len(r))
    return r

"""a = d[d['smoking_status'].isnull()]
a[a['stroke'] == 1]
d['smoking_status'].unique()"""


def plotagem(x,col):
    print("COLUNA: ",col)
    xs = x[col].unique()
    for i,vx in enumerate(xs):

        classe = x[ x[col] == vx ]
        p, n = len(strokes(classe,1)), len(strokes(classe,0))


        plt.bar([(i*2)],[n], color='b' )
        plt.bar([(i*2)+1],[p], color='r' )
        plt.title(vx)
        plt.show( )
        print(f"Porcentam dos positivados: {(p/(p+n) )*100:.2f}%")
    return

l = [
    "gender",
    "hypertension",
    "heart_disease",
    "ever_married",
    "Residence_type"
]

"""for i in l:
    plotagem(d,str(i))
    print("_------------------------------------------------------\n\n\n\n\n\n")
"""


'for i in l:\n    plotagem(d,str(i))\n    print("_------------------------------------------------------\n\n\n\n\n\n")\n'

In [100]:
"""
# "work_type_Private","work_type_Self-employed", "work_type_Govt_job",
plt.rcParams["figure.figsize"] = (30,30)

p = strokes(d,1)
n = strokes(d,0)

tp = [p[p['work'] == i] for i in range(1,4)]
tn = [n[n['work'] == i] for i in range(1,4)]

#lista = ['Private', 'Self-employed', 'Govt_job']
markers = ['s','+',0]

for i in range(3):
    plt.scatter(y=tn[i]['bmi'],x=tn[i]['avg_glucose_level'], c='g', alpha=0.1, marker=markers[i])
    plt.scatter(y=tp[i]['bmi'],x=tp[i]['avg_glucose_level'], c='r', alpha=1,  marker=markers[i])

#"smoking_status"]
"""

'\n# "work_type_Private","work_type_Self-employed", "work_type_Govt_job",\nplt.rcParams["figure.figsize"] = (30,30)\n\np = strokes(d,1)\nn = strokes(d,0)\n\ntp = [p[p[\'work\'] == i] for i in range(1,4)]\ntn = [n[n[\'work\'] == i] for i in range(1,4)]\n\n#lista = [\'Private\', \'Self-employed\', \'Govt_job\']\nmarkers = [\'s\',\'+\',0]\n\nfor i in range(3):\n    plt.scatter(y=tn[i][\'bmi\'],x=tn[i][\'avg_glucose_level\'], c=\'g\', alpha=0.1, marker=markers[i])\n    plt.scatter(y=tp[i][\'bmi\'],x=tp[i][\'avg_glucose_level\'], c=\'r\', alpha=1,  marker=markers[i])\n\n#"smoking_status"]\n'

In [101]:
"""print(len(strokes(d[d['smoking_status'].isnull()],1)))
print(len(strokes(d[d['smoking_status'].isnull()],0)))


print(len(strokes(d[d['smoking_status'] == 'smokes'],1)))
print(len(strokes(d[d['smoking_status']== 'smokes'],0)))"""


"print(len(strokes(d[d['smoking_status'].isnull()],1)))\nprint(len(strokes(d[d['smoking_status'].isnull()],0)))\n\n\nprint(len(strokes(d[d['smoking_status'] == 'smokes'],1)))\nprint(len(strokes(d[d['smoking_status']== 'smokes'],0)))"

In [102]:


dr = d['stroke']
d =d.drop('stroke',axis=1)
d =d.drop('id',axis=1)


x_tmp,x_valida,y_tmp,y_valida = train_test_split(d, dr,test_size= 0.1, random_state=0)
#x_treino,x_teste,y_treino,y_teste = train_test_split(x_tmp, y_tmp,test_size= 0.7, random_state=0)


from imblearn.over_sampling import RandomOverSampler
oversample = RandomOverSampler(sampling_strategy='minority')
oversample = RandomOverSampler(sampling_strategy=0.5)
X_over, y_over = oversample.fit_resample(x_tmp, y_tmp)




from sklearn.model_selection import train_test_split
x_treino,x_teste,y_treino,y_teste = train_test_split(X_over,y_over,test_size=0.3,stratify=y_over,random_state=42)






In [103]:


optimizer_adam = tf.keras.optimizers.Adam(learning_rate=0.0022)

optimizer_SGD = tf.keras.optimizers.SGD(
    learning_rate=0.02975,
    momentum=0.05,
    nesterov=False,
    name='SGD')

optimizer_Adagrad = tf.keras.optimizers.Adagrad(
    learning_rate=0.001,
    initial_accumulator_value=0.01,
    epsilon=1e-02,
    name="Adagrad",

)

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1,
                                                      write_images=True
                                                      )


model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(30,activation=tf.nn.relu,input_shape=(x_treino.shape[1],)))
model.add(tf.keras.layers.Dense(40,activation=tf.nn.relu ))
model.add(tf.keras.layers.Dense(20,activation=tf.nn.sigmoid ))
model.add(tf.keras.layers.Dense(1,activation=tf.nn.sigmoid ))
model.summary()
model.compile(optimizer=optimizer_adam,
              loss='MeanSquaredError',
              metrics=['accuracy','Recall'])

tf.summary.trace_on( profiler=False)

model.fit(x_treino,y_treino,batch_size=60, epochs=50,verbose=2,  validation_split=0.9,
          callbacks=[tensorboard_callback]
          )

teste  = model.evaluate(x_teste,  y_teste, verbose=2)
treino = model.evaluate(x_treino,  y_treino, verbose=2)

model.save('save/model2/'+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 30)                390       
                                                                 
 dense_21 (Dense)            (None, 40)                1240      
                                                                 
 dense_22 (Dense)            (None, 20)                820       
                                                                 
 dense_23 (Dense)            (None, 1)                 21        
                                                                 
Total params: 2,471
Trainable params: 2,471
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
68/68 - 3s - loss: 0.2223 - accuracy: 0.6197 - recall: 0.2213 - val_loss: 0.1884 - val_accuracy: 0.7312 - val_recall: 0.4627 - 3s/epoch - 40ms/step
Epoch 2/50
68/68 - 1s - l

Epoch 46/50
68/68 - 1s - loss: 0.1362 - accuracy: 0.8048 - recall: 0.7416 - val_loss: 0.1463 - val_accuracy: 0.7870 - val_recall: 0.7216 - 802ms/epoch - 12ms/step
Epoch 47/50
68/68 - 1s - loss: 0.1342 - accuracy: 0.8102 - recall: 0.7359 - val_loss: 0.1458 - val_accuracy: 0.7866 - val_recall: 0.7259 - 817ms/epoch - 12ms/step
Epoch 48/50
68/68 - 1s - loss: 0.1343 - accuracy: 0.8068 - recall: 0.7423 - val_loss: 0.1460 - val_accuracy: 0.7872 - val_recall: 0.7350 - 825ms/epoch - 12ms/step
Epoch 49/50
68/68 - 1s - loss: 0.1335 - accuracy: 0.8112 - recall: 0.7423 - val_loss: 0.1466 - val_accuracy: 0.7886 - val_recall: 0.7004 - 970ms/epoch - 14ms/step
Epoch 50/50
68/68 - 1s - loss: 0.1338 - accuracy: 0.8082 - recall: 0.7409 - val_loss: 0.1453 - val_accuracy: 0.7893 - val_recall: 0.7086 - 807ms/epoch - 12ms/step
540/540 - 1s - loss: 0.1423 - accuracy: 0.7945 - recall: 0.7090 - 574ms/epoch - 1ms/step
1259/1259 - 1s - loss: 0.1439 - accuracy: 0.7918 - recall: 0.7118 - 1s/epoch - 1ms/step
INFO:ten

In [104]:
print(model.evaluate(x_valida,  y_valida, verbose=2))

136/136 - 0s - loss: 0.1086 - accuracy: 0.8233 - recall: 0.6714 - 220ms/epoch - 2ms/step
[0.10857313126325607, 0.8232718706130981, 0.6714285612106323]


In [105]:
t = pd.read_csv("22.csv")

In [106]:
tt = Normaliza_dados(t)
tt = tt.drop('id',axis=1)
tt

,gender,age,hypertension,heart_disease,ever_married,Residence_type,avg_glucose_level,bmi,work_type_Private,work_type_Self-employed,work_type_Govt_job,work_type_Never_worked
0,0.0,0.80,0,0,0,0,0.117714,0.211,0,0,0,0
1,1.0,0.74,0,1,0,1,0.508163,0.260,1,0,0,0
2,1.0,0.14,0,0,1,1,0.163918,0.212,1,0,0,0
3,0.0,0.28,0,0,1,0,0.162286,0.234,1,0,0,0
4,1.0,0.63,0,0,0,1,0.116612,0.276,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
18596,0.0,0.20,0,0,1,1,0.079306,0.184,1,0,0,0
18597,0.0,0.61,0,0,0,1,0.638980,0.316,1,0,0,0
18598,1.0,0.79,0,0,0,1,0.288735,0.294,1,0,0,0
18599,0.0,0.55,0,0,0,1,0.059020,0.338,0,0,1,0


In [107]:
valores = model.predict(tt)
valores

582/582 [==============================] - 1s 1ms/step


array([[0.67177397],
       [0.80329454],
       [0.02159616],
       ...,
       [0.7362878 ],
       [0.11869537],
       [0.02354076]], dtype=float32)

In [132]:
val = [ 1 if x > 0.5 else 0 for x in valores]
print(sum(val)/len(val))
print("len:",len(val))

#df = 
df = t.copy()
df['stroke'] = val
df.drop(["gender","age","hypertension","heart_disease","ever_married","work_type","Residence_type","avg_glucose_level","bmi","smoking_status"],axis=1,inplace=True)
df.reset_index(drop=True, inplace=True)
df['id'].set_index=True


df.to_csv("final.csv",index=False)
df

0.1998279662383743
len: 18601


,id,stroke
0,36306,1
1,61829,1
2,14152,0
3,12997,0
4,40801,0
...,...,...
18596,67353,0
18597,362,1
18598,29839,1
18599,6438,0
